In [287]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data/', one_hot = True)
#the mnist data has been split into training validation and test data sets. It has also been preprocessed

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
#outline model
#we know we have 28x28 image so 28x28 = 784. Our input will be 784x1. Input units = 784
#output units are 10 as we know the umebrs given are 1-10
#we work with 2 hidden layers consisting of 50 units each

input_size = 784
output_size = 10
hidden_layer_size = 600

tf.reset_default_graph() #clears memory of all Variables left from previous tf runs

inputs_placeholder = tf.placeholder(tf.float32,[None,input_size])
targets_placeholder = tf.placeholder(tf.float32,[None,output_size])

weights_1 = tf.get_variable("weigths_1",[input_size,hidden_layer_size]) #if initialized this way, the default initializer is xavier
biases_1 = tf.get_variable("biases_1",[hidden_layer_size])

#get output for 1st hidden layer .... using relu as the non linearity/activation function
output_1 = tf.nn.relu(tf.matmul(inputs_placeholder,weights_1) + biases_1)

#time to create second layer
weigths_2 = tf.get_variable("weigths_2",[hidden_layer_size,hidden_layer_size])
biases_2 = tf.get_variable("biases_2", [hidden_layer_size])
output_2 = tf.nn.relu(tf.matmul(output_1,weigths_2) + biases_2)

#####if you want a 3rd layer
#3rd layer
#weigths_3 = tf.get_variable("weigths_3",[hidden_layer_size,hidden_layer_size])
#biases_3 = tf.get_variable("biases_3", [hidden_layer_size])
#output_3 = tf.nn.relu(tf.matmul(output_2,weigths_3) + biases_3)
#4th layer
#weigths_4 = tf.get_variable("weigths_4",[hidden_layer_size,hidden_layer_size])
#biases_4 = tf.get_variable("biases_4", [hidden_layer_size])
#output_4 = tf.nn.relu(tf.matmul(output_3,weigths_4) + biases_4)
#5th layer
#weigths_5 = tf.get_variable("weigths_5",[hidden_layer_size,hidden_layer_size])
#biases_5 = tf.get_variable("biases_5", [hidden_layer_size])
#output_5 = tf.nn.relu(tf.matmul(output_4,weigths_5) + biases_5)


#this is how we stack layers.
#now moving on to the output layer

weigths_11 = tf.get_variable("weigths_11",[hidden_layer_size,output_size])
biases_11 = tf.get_variable("biases_11", [output_size])
outputs = tf.matmul(output_2,weigths_11) + biases_11
#will incorporate the activation function for outputs in the loss as it is common practice

In [0]:
loss = tf.nn.softmax_cross_entropy_with_logits(logits = outputs, labels = targets_placeholder)
#the above function applies a softmax activation and calculates a cross entropy loss
#it is a numerically stable function. Obtaining very small number jeopardizes our model if we dont employ this
#not sure if required here but better to be safe 
#logits here means unscaled probabilities. we want our output layer to produce probabilities
#so the input into softmax is logit as the output of softmax gives probabilities

#we have the loss but mean loss gives performance boost
mean_loss = tf.reduce_mean(loss) #reduce_mean calculates the mean of a tensor

In [0]:
#now we choose the optimization method
learning_rate = 0.00002
optimize = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(mean_loss)

In [0]:
#to test accuracy
#as our output vector will give us probs. we want to get the highest prob of what it predicted and match 
#it to the actual target. if the indices of max of target and max of predicted is same, that means e predicted correct
#tf.argmax(outputs,1) gets the highest of each column(axis 1) in putputs vector
out_equals_target = tf.equal(tf.argmax(outputs,1),tf.argmax(targets_placeholder,1))
#out_equals_target is a vector containing 1 if accurately predcted else 0. the accuracy is the mean of this vector
accuracy = tf.reduce_mean(tf.cast(out_equals_target,tf.float32))

In [292]:
# time to LEARN
#first we initialize variabless
session = tf.InteractiveSession()
initializer = tf.global_variables_initializer()
session.run(initializer)
#then we do batching
batch_size = 100
batches_number = mnist.train._num_examples // batch_size #// means divide and give integer
max_epochs = 50

prev_validation_loss = 999999999. #giving it a high value just like max in java

for epoch_counter in range(max_epochs):
  curr_epoch_loss = 0
  for batch_counter in range(batches_number):
    input_batch,target_batch = mnist.train.next_batch(batch_size)
    _, batch_loss = session.run([optimize,mean_loss], 
                                feed_dict={inputs_placeholder:input_batch, targets_placeholder: target_batch})
    
    curr_epoch_loss +=batch_loss
    
  curr_epoch_loss /= batches_number #this is the training loss
  #now we will validate by forward propagating the validation set
  input_batch, target_batch = mnist.validation.next_batch(mnist.validation._num_examples)
  validation_loss, validation_accuracy = session.run([mean_loss,accuracy],
                                                     feed_dict={inputs_placeholder:input_batch, targets_placeholder: target_batch})
  print('Epoch ' + str(epoch_counter) + '. Training loss: ' + '{0:3f}'.format(curr_epoch_loss)
       + '. Validation loss: ' + '{0:3f}'.format(validation_loss)
       + '. Validation Accuracy: ' + '{0:2f}'.format(validation_accuracy *100) + '%')
  
  #eary stop rule
  if validation_loss > prev_validation_loss:
    break
    
  prev_validation_loss = validation_loss
  
print ('End of Training')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Epoch 0. Training loss: 1.122101. Validation loss: 0.475835. Validation Accuracy: 88.880002%
Epoch 1. Training loss: 0.392936. Validation loss: 0.313681. Validation Accuracy: 91.659999%
Epoch 2. Training loss: 0.299052. Validation loss: 0.260788. Validation Accuracy: 93.080002%
Epoch 3. Training loss: 0.255414. Validation loss: 0.230134. Validation Accuracy: 93.839997%
Epoch 4. Training loss: 0.226506. Validation loss: 0.205857. Validation Accuracy: 94.340003%
Epoch 5. Training loss: 0.204360. Validation loss: 0.189669. Validation Accuracy: 94.819999%
Epoch 6. Training loss: 0.187024. Validation loss: 0.175249. Validation Accuracy: 95.120001%
Epoch 7. Training loss: 0.172230. Validation loss: 0.163257. Validation Accuracy: 95.380002%
Epoch 8. Training loss: 0.159559. Validation loss: 0.153420. Validation Accuracy: 95.639998%
Epoch 9. Training loss: 0.148523. Validation loss: 0.147079. Validation Accuracy: 96.120000%
Epoch 10. Training loss: 0.139003. Validation loss: 0.138761. Validati

In [293]:
#time to test using test data set

input_batch,target_batch = mnist.test.next_batch(mnist.test._num_examples) #get input and outputs of test data
test_accuracy = session.run([accuracy], feed_dict = {inputs_placeholder:input_batch, targets_placeholder: target_batch})
print('{0:3f}'.format(test_accuracy[0]*100) + '%')

97.710001%


In [0]:
#final accuracy is as shown above

**Initial Hyperparams:**

input_size = 784, 
output_size = 10, 
hidden_layer_size = 50, 
learning_rate = 0.001, 
batch_size = 100

TIME TAKEN: 15s

ACCURACY 97.10%



**1. The *width* (the hidden layer size) of the algorithm. Try a hidden layer size of 200. How does the validation accuracy of the model change? What about the time it took the algorithm to train? Can you find a hidden layer size that does better?**

TIME TAKEN: 15s

ACCURACY 97.77%

The validation accuracy is significantly higher (as the algorithm with 50 hidden units was too simple of a model).

Naturally, it takes the algorithm much longer to train (unless early stopping is triggered too soon).

A hidden layer size of 500 (and not only) works better.


**2. The *depth* of the algorithm. Add another hidden layer to the algorithm. This is an extremely important exercise! How does the validation accuracy change? What about the time it took the algorithm to train? Hint: Be careful with the shapes of the weights and the biases.**

With 4 layers, time taken was 12s and accuracy was 97.03%


**3. The *width and depth* of the algorithm. Add as many additional layers as you need to reach 5 hidden layers. Moreover, adjust the width of the algorithm as you find suitable. How does the validation accuracy change? What about the time it took the algorithm to train?**

With 4 layers, time taken was 10s and accuracy was 96.16%

With 4 layers, time taken was 10s and accuracy was 96.44%

**4. Fiddle with the activation functions. Try applying sigmoid transformation to both layers. The sigmoid activation is given by the method: tf.nn.sigmoid()**

Time taken: 19 s

Accuracy: 96.8%. on average took more epochs to train.

Generally, we should reach an inferior solution. That is because relu 'cleans' the noise in the data

**5. Fiddle with the activation functions. Try applying a ReLu to the first hidden layer and tanh to the second one. The tanh activation is given by the method: tf.nn.tanh()**

TIME TAKEN: 13s

ACCURACY 97.15%

**6. Adjust the batch size. Try a batch size of 1000. How does the required time change? What about the accuracy?**

TIME TAKEN: 13s

ACCURACY 96%. Did not trigger early stop in the 15 epochs. The accuracy in the limited  epochs was lower than in batch size 100. A bigger batch size results in slower training

**7. Adjust the batch size. Try a batch size of 1. That's the SGD. How do the time and accuracy change? Is the result coherent with the theory?**

TIME TAKEN: 125s

ACCURACY 96%.  triggered early stop in the the 2nd epoch. Took a minute approx each for each epoch. the accuracy was compritively lower too

**8. Adjust the learning rate. Try a value of 0.0001. Does it make a difference?**

TIME TAKEN: 21s

ACCURACY 95.4%. The learning rate was too small and hence it gradually the accuracy was improving but in the 15 epochs it still did not come close to as accurate our previous models have been. Since the training is lower, I have also changed the max_epochs to 50, otherwise early stopping would be triggered.

The result is basically the same, but we reach it much slower.

**9. Adjust the learning rate. Try a value of 0.02. Does it make a difference?**

TIME TAKEN: 7s

ACCURACY 94%. The learning rate was high and accuracy was lower

**The final version i have is by playing around with the hyper params. Not necessarily the best accuracy wise, but was good for practice**

input_size = 784, 
output_size = 10, 
hidden_layer_size = 600, 
learning_rate =0.00002, 
batch_size = 100

ACCURACY 97.71%
